In [1]:
import pandas as pd
import requests

In [2]:
# chooses only a specific sheet from the Excel file
df = pd.read_excel('AidDatasGlobalChineseDevelopmentFinanceDataset_v3.0.xlsx', sheet_name='GCDF_3.0')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20985 entries, 0 to 20984
Columns: 126 entries, AidData Record ID to Loan Detail Score
dtypes: datetime64[ns](7), float64(24), int64(7), object(88)
memory usage: 20.2+ MB


In [3]:
# filters data points to only show 'Asia' under 'Recipient Region'
df = df[df['Recipient Region'] == 'Asia']
df.head(3)

,AidData Record ID,Recommended For Aggregates,AidData Parent ID,Umbrella,Financier Country,Recipient,Recipient ISO-3,Recipient Region,Commitment Year,Implementation Start Year,...,OECD ODA Income Group,Location Narrative,Geographic Level of Precision Available,ADM1 Level Available,ADM2 Level Available,Geospatial Feature Available,Source Quality Score,Data Completeness Score,Implementation Detail Score,Loan Detail Score
0,94556,Yes,NaN,No,China (People's Republic of),Afghanistan,AFG,Asia,2021,2021.0,...,Low income,No location info available.,NaN,NaN,NaN,NaN,5,3,4.0,NaN
1,94564,Yes,NaN,No,China (People's Republic of),Afghanistan,AFG,Asia,2021,2021.0,...,Low income,NaN,NaN,NaN,NaN,NaN,5,3,4.0,NaN
2,94565,Yes,NaN,No,China (People's Republic of),Afghanistan,AFG,Asia,2021,2021.0,...,Low income,Kabul Afghanistan,NaN,NaN,NaN,NaN,4,3,5.0,NaN


In [4]:
# shows list of 'Recipient' and number of unique records
df.groupby('Recipient')['AidData Record ID'].nunique()

Recipient
Afghanistan                              169
Armenia                                   52
Asia, regional                            54
Azerbaijan                                53
Bangladesh                               175
Brunei Darussalam                         53
Cambodia                                 418
Democratic People's Republic of Korea    148
Georgia                                   52
India                                    113
Indonesia                                437
Kazakhstan                               181
Kyrgyz Republic                          146
Lao People's Democratic Republic         346
Malaysia                                 176
Maldives                                 101
Mongolia                                 232
Myanmar                                  495
Nepal                                    222
Pakistan                                 496
Philippines                              267
Sri Lanka                                301


In [5]:
# filtering non-Southeast Asia rows

exclude_rows = ['Afghanistan', 'Armenia', 'Azerbaijan', "Democratic People's Republic of Korea", 'Georgia', 'India', 'Kazakhstan', 'Kyrgyz Republic', 'Maldives', 'Mongolia', 'Nepal', 'Pakistan', 'Sri Lanka', 'Tajikistan', 'Turkmenistan', 'Uzbekistan']
filtered_df = df[~df['Recipient'].str.contains('|'.join(exclude_rows))]

In [6]:
filtered_df.groupby('Recipient')['AidData Record ID'].nunique()

Recipient
Asia, regional                       54
Bangladesh                          175
Brunei Darussalam                    53
Cambodia                            418
Indonesia                           437
Lao People's Democratic Republic    346
Malaysia                            176
Myanmar                             495
Philippines                         267
Thailand                            116
Timor-Leste                         131
Viet Nam                            191
Name: AidData Record ID, dtype: int64

In [7]:
# saves data filtered 'Asia', then Southeast Asian countries
# use this only as some sort of checkpoint
filtered_df.to_csv('data.csv')

In [80]:
# finding a string or keyword in all cells
# keep only rows that have the search keywork

search_string = 'petrochemical'

def check_string(cell):
    return search_string in str(cell)

mask = filtered_df.map(check_string).any(axis=1)
new_df = filtered_df[mask]

In [76]:
new_df.dtypes

AidData Record ID                 int64
Recommended For Aggregates       object
AidData Parent ID                object
Umbrella                         object
Financier Country                object
                                 ...   
Geospatial Feature Available     object
Source Quality Score              int64
Data Completeness Score           int64
Implementation Detail Score     float64
Loan Detail Score               float64
Length: 126, dtype: object

In [77]:
## FIRST BATCH OF COLUMNS TO DROP due to use of indices

## columns to exclude by name
# exclude_columns = ['Recipient ISO-3', 'Recipient Region', 'Implementation Start Year', 'Completion Year', 'Flow Type', 'OECD ODA Concessionality Threshold', 'Flow Class', 'Sector Code', 'COVID', 'On-lending', 'Guarantee Provided', 'Guarantor', 'Guarantor Agency Type', 'Insurance Provided', 'Insurance Provider', 'Insurance Provider Agency Type', 'Collateralized', 'Collateral Provider', 'Collateral Provider Agency Type', 'Security Agent/Collateral Agent', 'Security Agent/Collateral Agent Type', 'Collateral']

## columns to exclude by index
# column_indices = list(range(49, 58)) + list(range(63, 126))

## convert indices to column names
# exclude_by_index = [df.columns[i] for i in column_indices]

## combine the two lists
# exclude_columns = exclude_columns + exclude_by_index

## drop the columns
# new_df = new_df.drop(columns=exclude_columns, errors='ignore')

In [81]:
# BUT, for-loops are illegal** (an inside joke)

new_df = new_df.drop(columns=new_df.columns[66:126])
new_df = new_df.drop(columns=new_df.columns[56:57])
new_df = new_df.drop(columns=new_df.columns[46:54])

# columns to exclude by name
exclude_columns = ['Recipient ISO-3', 'Recipient Region', 'Implementation Start Year', 'Completion Year', 'Flow Type', 'OECD ODA Concessionality Threshold', 'Flow Class', 'Sector Code', 'COVID', 'On-lending', 'Guarantee Provided', 'Guarantor', 'Guarantor Agency Type', 'Insurance Provided', 'Insurance Provider', 'Insurance Provider Agency Type', 'Collateralized', 'Collateral Provider', 'Collateral Provider Agency Type', 'Security Agent/Collateral Agent', 'Security Agent/Collateral Agent Type', 'Collateral']
new_df = new_df.drop(columns=exclude_columns, errors='ignore')

In [82]:
new_df.dtypes

AidData Record ID                                            int64
Recommended For Aggregates                                  object
AidData Parent ID                                           object
Umbrella                                                    object
Financier Country                                           object
Recipient                                                   object
Commitment Year                                              int64
Title                                                       object
Description                                                 object
Staff Comments                                              object
Status                                                      object
Intent                                                      object
Flow Type Simplified                                        object
Sector Name                                                 object
Infrastructure                                              ob

In [83]:
# SECOND BATCH OF COLUMNS TO DROP, using range of strings / column names

# finds columns ranging between identified strings, then drops them
# does not work for multiple ranges; list is better
new_df.drop(new_df.loc[:, 'Recommended For Aggregates':'Financier Country'], axis=1, inplace=True)

# `inplace=True` will return a new df with the specified columns dropped
# if used, no need to save / overwrite df with `new_df = new_df.drop ...`

In [84]:
new_df.dtypes

AidData Record ID                                            int64
Recipient                                                   object
Commitment Year                                              int64
Title                                                       object
Description                                                 object
Staff Comments                                              object
Status                                                      object
Intent                                                      object
Flow Type Simplified                                        object
Sector Name                                                 object
Infrastructure                                              object
Funding Agencies                                            object
Funding Agencies Type                                       object
Co-financed                                                 object
Co-financing Agencies                                       ob

In [85]:
# THIRD BATCH: drop column if commitment year is later than 2016
# new_df.drop(columns=new_df.loc[(new_df['Commitment Year'] > 2016)].columns)

# BUT, we are not dropping COLUMNS where commitment year > 2016. 
# What we want is to drop ROWS.

# Hence: 

new_df = new_df[new_df['Commitment Year'] > 2016]

In [86]:
new_df.head(2)

,AidData Record ID,Recipient,Commitment Year,Title,Description,Staff Comments,Status,Intent,Flow Type Simplified,Sector Name,...,Adjusted Amount (Nominal USD),Commitment Date (MM/DD/YYYY),Commitment Date Estimated,Planned Implementation Start Date (MM/DD/YYYY),Actual Implementation Start Date (MM/DD/YYYY),Actual Implementation Start Date Estimated,Deviation from Planned Implementation Start Date,Planned Completion Date (MM/DD/YYYY),Actual Completion Date (MM/DD/YYYY),Actual Completion Date Estimated
4036,89494,Malaysia,2019,Bank of China contributes to $5.65 billion syn...,"On March 28, 2018, Pengerang Refining Company ...",1. This project is also known as the Refinery ...,Implementation,Mixed,Loan,"INDUSTRY, MINING, CONSTRUCTION",...,282500000.0,2019-11-05,No,NaT,NaT,NaN,NaN,NaT,NaT,NaN
4037,89495,Malaysia,2019,ICBC contributes to $5.65 billion syndicated l...,"On March 28, 2018, Pengerang Refining Company ...",1. This project is also known as the Refinery ...,Implementation,Mixed,Loan,"INDUSTRY, MINING, CONSTRUCTION",...,282500000.0,2019-11-05,No,NaT,NaT,NaN,NaN,NaT,NaT,NaN


In [87]:
new_df.to_csv('data_petrochemical.csv')